### 9.3 Exercise: Neural Network Classifiers
1. Neural Network Classifier with Scikit
Using the multi-label classifier dataset (categorized-comments.jsonl), fit a neural network classifier using scikit-learn to predict the comment category. Use the code found in chapter 12 of the Applied Text Analysis with Python book as a guide, but you will need to modify the code for this dataset. Report the accuracy, precision, recall, F1-score, and confusion matrix.
2. Neural Network Classifier with Keras
Using the multi-label classifier dataset (categorized-comments.jsonl), fit a neural network classifier with Keras to predict the comment category. Use the code found in chapter 12 of the Applied Text Analysis with Python book as a guide, but you will need to modify the code for this dataset. Report the accuracy, precision, recall, F1-score, and confusion matrix.
3. Classifying Images
In chapter 20 of the Machine Learning with Python Cookbook, implement the code found in section 20.15 classify MSINT images using a convolutional neural network. Report the accuracy of your results.

#### 1. Neural Network Classifier with Scikit

In [39]:
import pandas as pd
import numpy as np
from functools import wraps
import joblib
import sklearn
from sklearn.model_selection import cross_val_score

In [25]:
# Import the file
df = pd.read_json('categorized-comments.jsonl', lines = True)

In [101]:
# Check the header
df.head()

,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


In [27]:
# Check the shape
df.shape

(606476, 2)

In [31]:
def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        return result, time.time() - start
    return wrapper

def documents(corpus):
    return list(corpus.reviews())

def continuous(corpus):
    return list(corpus.scores())

def make_categorical(corpus):
  
    terrible : 0.0 < y <= 3.0
    okay     : 3.0 < y <= 5.0
    great    : 5.0 < y <= 7.0
    amazing  : 7.0 < y <= 10.1


    return np.digitize(continuous(corpus), [0.0, 3.0, 5.0, 7.0, 10.1])

In [130]:
def train_model(path, model, continuous = True, saveto = None, cv = 12):
    # Load the corpus data and labels for classification
    corpus = df
    X = documents(corpus)
    if continuous:
        y = continuous(corpus)
        scoring = 'r2_score'
    else:
        y = make_categorical(corpus)
        scoring = 'f1_score'

    # Compute cross validation scores
    scores = cross_val_score(model, X, y, cv = cv, scoring = scoring)

    # Fit the model on entire data set
    model.fit(X, y)

    # Write to disk if specified
    if saveto:
        joblib.dump(model, saveto)

    # Return scores as well as training time via decorator
    return scores



In [150]:
from transformer import TextNormalizer 
from reader import PickledReviewsReader 
from sklearn.pipeline import Pipeline 
from sklearn.neural_network import MLPRegressor, MLPClassifier 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus.reader.api import CorpusReader

# I'm at a total loss on how to resolve the issues with the imports.  
# I tried all the updates from book's github page with no success
# I've ried uninstallin/reinstalling everything I can think of, but nothing is working 
    # (even the suggestions from classmates when I asked for help in the chat)
    # I can install the main library, but I get errors that the specified library isn't in the module
    # For example TextNormalizer isn't found in Transformer
# Unfortunatly, it affects the rest of the exercise, but I tried to go as far with the code examples as I could
# I'm truly at a loss how to make it work

ModuleNotFoundError: No module named 'transformer'

In [129]:
cpath = '../review_corpus_proc' 
regressor = Pipeline([('norm', TextNormalizer()), 
                       ('tfidf', TfidfVectorizer()), 
                       ('ann', MLPRegressor(hidden_layer_sizes = [500,150], verbose = True))
                      ]) 

regression_scores = train_model(cpath, regressor, continuous = True ) 

classifier = Pipeline([ 
    ('norm', TextNormalizer()), 
    ('tfidf', TfidfVectorizer()), 
    ('ann', MLPClassifier( hidden_layer_sizes = [500 , 150], verbose = True)) 
]) 

classifer_scores = train_model(cpath, classifier, continuous = False) 

NameError: name 'TextNormalizer' is not defined

In [126]:
cpath = '../review_corpus_proc' 

mpath = { 
    'keras_model' : 'keras_nn.h5',
    'sklearn_pipe' : 'pipeline.pkl' 
} 

scores = train_model(cpath, pipeline, saveto = mpath, cv = 12) 
  

NameError: name 'pipeline' is not defined

#### 2. Neural Network Classifier with Keras

In [89]:
# Import the libraries
from keras.layers import Dense
from keras.models import Sequential


In [98]:
# Import the file
df = pd.read_json('categorized-comments.jsonl', lines = True)

# Check the header
df.head()

,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


In [105]:
# Set the features and classes
N_FEATURES = 5000 
N_CLASSES = 4 


In [106]:
# Create the function for a complied neural network
def build_network(): 
    nn = Sequential() 
    nn.add(Dense(500, activation = 'relu', input_shape = (N_FEATURES,))) 
    nn.add(Dense(150, activation = 'relu')) 
    nn.add(Dense(N_CLASSES, activation = 'softmax')) 
    nn.compile(loss = 'categorical_crossentropy', 
               optimizer = 'adam', 
               metrics = ['accuracy']
              ) 
    return nn 
  

In [137]:
if __name__ == '__main__' : 
    from sklearn.pipeline import Pipeline 
    from transformer import TextNormalizer 
    from keras.wrappers.scikit_learn import KerasClassifier 
    from sklearn.feature_extraction.text import TfidfVectorizer 
    
    pipeline = Pipeline([ 
        ( 'norm', TextNormalizer()), 
        ( 'vect', TfidfVectorizer( max_features = N_FEATURES)), 
        ( 'nn', KerasClassifier(build_fn = build_network,
                                  epochs = 200, 
                                  batch_size = 128)) 
    ]) 


# This is sthe same issue I had in the exercise above with the imports
# Unfortunatly, it affects the rest of the exercise    
# Although I did try as much of the code as I could

ModuleNotFoundError: No module named 'transformer'

In [138]:
# Train the model and fit the data
def train_model(path, model, saveto = None, cv = 12): 
    corpus = PickledReviewsReader(path) 
    X = documents(corpus) 
    y = make_categorical(corpus) 
    
    scores = cross_val_score(model, X, y, cv = cv, scoring = 'accuracy', n_jobs = -1) 
    model.fit(X, y ) 
    
    if saveto: 
        model.steps[1][1].model.save(saveto['keras_model']) 
        model.steps.pop(-1) 
        joblib.dump(model,saveto['sklearn_pipe']) 
        
    return scores 


#### 3. Classifying Images

In [ ]:
# Import the libraries
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [55]:
# Set that the color channel value will be first
K.set_image_data_format("channels_last")

# Set seed
np.random.seed(0)

# Set image information
height = 28
width = 28
channels = 1

In [56]:
# Load data and target from MNIST data
(data_train, target_train), (data_test, target_test) = mnist.load_data()

# Reshape training image data into features
data_train = data_train.reshape(data_train.shape[0], height, width, channels)

# Reshape test image data into features
data_test = data_test.reshape(data_test.shape[0], height, width, channels)

# Rescale pixel intensity to between 0 and 1
features_train = data_train / 255
features_test = data_test / 255

In [57]:
# One-hot encode target
target_train = np_utils.to_categorical(target_train)
target_test = np_utils.to_categorical(target_test)
number_of_classes = target_test.shape[1]

In [58]:
# Start neural network
network = Sequential()

# Add convolutional layer with 64 filters, a 5x5 window, and ReLU activation function
network.add(Conv2D(filters = 64,
                   kernel_size = (5, 5),
                   input_shape = (width, height, channels),
                   activation = 'relu'))

# Add max pooling layer with a 2x2 window
network.add(MaxPooling2D(pool_size = (2, 2)))

# Add dropout layer
network.add(Dropout(0.5))

# Add layer to flatten input
network.add(Flatten())

# # Add fully connected layer of 128 units with a ReLU activation function
network.add(Dense(128, activation = "relu"))

# Add dropout layer
network.add(Dropout(0.5))

# Add fully connected layer with a softmax activation function
network.add(Dense(number_of_classes, activation = "softmax"))

# Compile neural network
network.compile(loss = "categorical_crossentropy", # Cross-entropy
                optimizer = "rmsprop", # Root Mean Square Propagation
                metrics = ["accuracy"]) # Accuracy performance metric


In [61]:
# Train neural network
network.fit(features_train, # Features
            target_train, # Target
            epochs = 2, # Number of epochs
            # verbose = 0, # Don't print description after each epoch
            batch_size = 1000, # Number of observations per batch
            validation_data = (features_test, target_test)) # Data for evaluation Using TensorFlow backend.

Epoch 1/2
60/60 [==============================] - 17s 284ms/step - loss: 0.1258 - accuracy: 0.9636 - val_loss: 0.0603 - val_accuracy: 0.9806
Epoch 2/2
60/60 [==============================] - 17s 277ms/step - loss: 0.0972 - accuracy: 0.9705 - val_loss: 0.0486 - val_accuracy: 0.9832


#### Accuracy:  with scores of 96 and 97, I think these results are very accurate.  